# Quaternions representing rotations SO(3)
This notebook is a short sumamry of how SO(3) is handled in Pinocchio, and mostly quaternion representation. 
## SO(3), Euler angles, rotations

In [1]:
import pinocchio
from pinocchio import SE3, Quaternion
import numpy as np
from numpy.linalg import norm
M = SE3.Random()
R = M.rotation
p = M.translation
print('R =\n' + str(R))
print('p =\n' + str(p.T))

R =
[[ 0.0452059  -0.99506743 -0.08830191]
 [-0.03981301  0.08652748 -0.99545362]
 [ 0.99818403  0.04851594 -0.03570508]]
p =
[0.02680182 0.90445945 0.83239014]


A rotation is simply a 3x3 matrix. It has a unit norm:

In [2]:
print(R @ R.T)

[[ 1.00000000e+00  2.44135006e-16 -2.43295690e-16]
 [ 2.44135006e-16  1.00000000e+00  1.78876415e-16]
 [-2.43295690e-16  1.78876415e-16  1.00000000e+00]]


It can be equivalently represented by a quaternion. Here we have made the choice to create a specific class for quaternions (i.e. they are not vectors, and can be e.g. multiplied), but you can get the 4 coefficients with the adequate method. Note that the corresponding vector is also of norm 1.

In [3]:
quat = Quaternion(R)
print(norm(quat.coeffs()))

1.0000000000000002


Angle-axis representation are also implemented in the class AngleAxis.

In [4]:
from pinocchio import AngleAxis
utheta = AngleAxis(quat)
print(utheta.angle, utheta.axis)

2.039786642710466 [ 0.58516826 -0.60899964  0.53544144]


You can display rotation in the viewer.

In [8]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer()
viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [9]:
viz.addBox('world/box', [.1, .2, .3], [1, 0, 0, 1])
viz.applyConfiguration('world/box', [.1, .2, .3] + list(quat.coeffs()))

### Quaternion you said?
Quaternions are "complex of complex", introduced form complex as complex are from reals. Let's try to understand what they contains in practice. Quaternions are of the form w+xi+yj+zk, with w,x,y,z real values, and i,j,k the 3 imaginary numbers. We store them as 4-d vectors, with the real part first: quat = [x,y,z,w]. We can interprete w as encoding the cosinus of the rotation angle. Let's see that.

In [10]:
from numpy import arccos
print(arccos(quat[3]))
print(AngleAxis(quat).angle)

1.019893321355233
2.039786642710466


Indeed, w = cos(θ/2). Why divided by two? For that, let's see how the quaternion can be used to represent a rotation. We can encode a 3D vector in the imaginary part of a quaternion.

In [11]:
p = np.random.rand(3)
qp = Quaternion(0., p[0], p[1], p[2])
print(qp)

(x,y,z,w) = 0.230725 0.815279 0.536544        0



The real product extends over quaternions, so let's try to multiply quat with p:

In [12]:
print(quat * qp)

(x,y,z,w) = -0.529588  0.264508  0.807073 0.0632258



Well that's not a pure imaginary quaternion anymore. And the imaginary part does not contains somethig that looks like the rotated point:

In [13]:
print(quat.matrix() @ p)

[-0.84820525 -0.47274614  0.25070236]


The pure quaternion is obtained by multiplying again on the left by the conjugate (w,-x,-y,-z).

In [14]:
print(quat * qp * quat.conjugate())

(x,y,z,w) = -0.848205 -0.472746  0.250702         0



That is a pure quaternion, hence encoding a point, and does corresponds to R*p. Magic, is it not? We can prove that the double product of quaternion does corresponds to the rotation. Indeed, a quaternion rather encode an action (a rotation) in $R^4$, but which moves our point p outside of $R^3$. The conjugate rotation brings it back in $R^3$ but applies a second rotation. Since we rotate twice, it is necessary to apply only half of the angle each time.
What if we try to apply the rotation quat on the imaginary part of the quaternion?

In [15]:
qim = Quaternion(quat)  # copy
qim[3] = 0
print(qim, quat * qim * quat.conjugate())

(x,y,z,w) =  0.498594 -0.518899  0.456224         0
 (x,y,z,w) =    0.498594   -0.518899    0.456224 5.55112e-17



What kind of conclusion can we get from this? What geometrical interpretation can we give to $q_{im}$? What about $||q_{im}||$?

### The SLERP example
Let's practice! Implement a linear interpolation between two position p0 and p1, i.e. find the position p(t) with t varying from 0 to 1, with p(0)=p0, p(1)=p1 and continuity between the two extrema.

In [20]:
# %load appendix/solution_lerp.py
import time
import numpy as np

p0 = np.random.rand(3)
p1 = np.random.rand(3)

for t in np.arange(0, 1, .01):
    p = p0 * (1 - t) + p1 * t
    viz.applyConfiguration('world/box', list(p) + list(quat.coeffs()))
    time.sleep(.01)


LERP with quaternions is not working because they are not normalize. Instead we can take either the normalization of the LERP (NLERP), or the spherical LERP (SLERP). 

In [22]:
# %load appendix/solution_slerp.py
from time import sleep
import numpy as np
from numpy.linalg import norm
from pinocchio import SE3, AngleAxis, Quaternion

def _lerp(p0, p1, t):
    return (1 - t) * p0 + t * p1

def slerp(q0, q1, t):
    assert (t >= 0 and t <= 1)
    a = AngleAxis(q0.inverse() * q1)
    return Quaternion(AngleAxis(a.angle * t, a.axis))

def nlerp(q0, q1, t):
    q0 = q0.coeffs()
    q1 = q1.coeffs()
    lerp = _lerp(q0, q1, t)
    lerp /= norm(lerp)
    return Quaternion(lerp[3], *list(lerp[:3]))

q0 = Quaternion(SE3.Random().rotation)
q1 = Quaternion(SE3.Random().rotation)
viz.applyConfiguration('world/box', [0, 0, 0] + list(q0.coeffs()))
sleep(.1)
for t in np.arange(0, 1, .01):
    q = nlerp(q0, q1, t)
    viz.applyConfiguration('world/box', [0, 0, 0] + list(q.coeffs()))
    sleep(.01)
sleep(.1)
viz.applyConfiguration('world/box', [0, 0, 0] + list(q1.coeffs()))



## Cheat sheet

You can convert quaternion to rotation matrix and create SE3 objects as follows:


In [23]:
qu = Quaternion(.7,.2,.2,.6)# Quaternion: take care that norm <= 1 (and approx 1)
R  = qu.matrix()                # Create a rotation matrix from quaternion
p  = np.array([0.,0.,0.77])     # Translation (R3) vector)
M  = SE3(R,p)                   # Create a nomogeneous matrix from R,P

# Typical tool position
from pinocchio.utils import rotate
M = SE3(rotate('z',1.)@rotate('x',.2), np.array([0.1,0.02,.65]))